### 2. 소속 가수 앨범 날짜 크롤링

In [6]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import pandas as pd
import re

In [7]:
# driver_path = 'C:/Users/Playdata/Desktop/dev/setup/chromdriver.exe'
driver_path = 'C:/Users/k9942/OneDrive/바탕 화면/playdata/dev/setup/chromdriver.exe'
driver = webdriver.Chrome(driver_path)

C:\Users\k9942\AppData\Local\Temp\ipykernel_10000\3454815965.py:3: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(driver_path)


In [8]:
# 앨범 정보 크롤링 함수
def create_album_info(url, singer):
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    album_info = soup.select('div > .atist_info')[1:]

    album_total_data = []

    for album in album_info:
        title = album.select_one('div > dl > dt > a').get_text()
        type = album.select_one('.vdo_name').get_text()[1:-1]
        release_date = album.select_one('.cnt_view').get_text()
        album_data = [singer, title, type, release_date]
        album_total_data.append(album_data)
    
    return album_total_data


In [9]:
# 페이지 인덱스로 url 생성하는 함수
def get_url(idx):
    url = 'https://www.melon.com/artist/album.htm?artistId=672375#params%5BlistType%5D=0&params%5BorderBy%5D=ISSUE_DATE&params%5BartistId%5D=672375&po=pageObj&startIndex=' + f'{idx}'
    return url

In [10]:
# 앨범 정보 크롤링
bts_total_data = []
page_num = 6
index = [i*15+1 for i in range(page_num)]

for idx in index:
    url = get_url(idx)
    singer = 'BTS'
    bts_album_data = create_album_info(url, singer)
    bts_total_data.append(bts_album_data)

In [12]:
import numpy as np 
columns = ['singer', 'title', 'album_type', 'release_date']
df = pd.DataFrame(np.array(bts_total_data, dtype='object').reshape(-1,4))
# df.reset_index(drop=True, inplace=True)
df.columns = columns
df.tail()

,singer,title,album_type,release_date
85,BTS,DARK&WILD,정규,2014.08.20
86,BTS,Skool Luv Affair Special Addition,정규,2014.05.14
87,BTS,Skool Luv Affair,EP,2014.02.12
88,BTS,"O!RUL8,2?",EP,2013.09.11
89,BTS,2 COOL 4 SKOOL,싱글,2013.06.12


In [122]:
# df.to_csv('data/BTS_album.csv', index=False)

### 수정 1
참조 테이블인 stock 의 date의 범위는 2020-10-15 부터인데 BTS 앨범 정보는 2013년도 부터 있기 때문에 참조할 데이터가 없어 db에 데이터를 insert 할 시에 오류가 난다. 때문에, 2010-10-15 이후에 발매한 앨범만 추출해야한다.

In [13]:
# 문자열을 datetime으로 변환하기
df['release_date'] = pd.to_datetime(df['release_date'], format='%Y-%m-%d')

In [14]:
bts_df = df[df['release_date'].between('2020-10-15', '2023-05-25')]

In [56]:
# bts_df.to_csv('data/BTS_album.csv', index=False)

In [58]:
bts_df.query('release_date.dt.dayofweek == 6')
# 문제점: 주가 데이터는 공휴일 및 주말에 없기 때문에 참조 상에 문제가 생긴다. => db 변경

,singer,title,album_type,release_date
3,BTS,Dreamers [Music from the FIFA World Cup Qatar ...,싱글,2022-11-20
18,BTS,Dreamers [Music from the FIFA World Cup Qatar ...,싱글,2022-11-20
35,BTS,My Universe (SUGA's Remix),리믹스,2021-09-26


### 수정 2

's (작은 따옴표) 때문에 쿼리 Syntax 오류가 난다.
-> 앞에 작은 따옴표를 하나 더 붙여서 이스케이프를 해야한다.

In [61]:
bts_df.loc[35]

singer                                 BTS
title           My Universe (SUGA's Remix)
album_type                             리믹스
release_date           2021-09-26 00:00:00
Name: 35, dtype: object

In [4]:
# 아이디어
"My Universe (SUGA's Remix)".replace("'", "''")

"My Universe (SUGA''s Remix)"

In [16]:
bts_df['title'] = [title.replace("'", "''") for title in bts_df['title']]

C:\Users\k9942\AppData\Local\Temp\ipykernel_10000\1275685817.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bts_df['title'] = [title.replace("'", "''") for title in bts_df['title']]


In [17]:
# 확인
bts_df.loc[35]

singer                                  BTS
title           My Universe (SUGA''s Remix)
album_type                              리믹스
release_date            2021-09-26 00:00:00
Name: 35, dtype: object

In [18]:
bts_df.to_csv('data/BTS_album.csv', index=False)